In [2]:
import os,sys
sys.path.append("/home/doyooni303/experiments/recsys/ReLLMRec")
import argparse
import yaml
import hashlib
import torch
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.optim import AdamW

from src.models import CandiRec
from src.train import Trainer
from src.utils import set_seed

In [34]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
    
parser = argparse.ArgumentParser()
parser.add_argument("--path", type=str, default="/data3/doyoon/sequential_recsys/datasets/amazon/Movies_and_TV/")
parser.add_argument("--type", type=str, default="Original")
parser.add_argument("--fname", type=str, default="Movies_and_TV")
parser.add_argument("--gpu", type=str, default="0,1,2,3", help="GPU ids to use (comma separated)")
parser.add_argument("--top_k", type=int, default=10)
parser.add_argument("--train_batch_size", type=int, default=4)
parser.add_argument("--eval_batch_size", type=int, default=8)
parser.add_argument("--max_epochs", type=int, default=10)
parser.add_argument("--max_steps", type=int, default=100)
parser.add_argument("--train_mode", type=str, default="epochs", help="[epochs, steps]")

args = parser.parse_args([])

# Set CUDA_VISIBLE_DEVICES based on gpu argument
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

yaml_path = os.path.join("/home/doyooni303/experiments/recsys/ReLLMRec/configs", f"{args.fname}.yaml")
config = yaml.load(open(yaml_path, "r"), Loader=yaml.FullLoader)

for key, value in args.__dict__.items():
    config.update({key: value})

if config["train_mode"] == "epochs":
    name = f"-".join([f"{key}_{config[key]}" for key in ["top_k", "max_epochs"]])
    config.update({"eval_interval": config["max_epochs"] // 10})
elif config["train_mode"] == "steps":
    name = f"-".join([f"{key}_{config[key]}" for key in ["top_k", "max_steps"]])
    config.update({"eval_interval": config["max_steps"] // 10})


In [35]:
# Create dataset
from src.dataset import AmazonDataset
train_dataset = AmazonDataset(config, "train")


splitting data by user: 100%|██████████| 716592/716592 [00:03<00:00, 215493.41it/s]


In [37]:
from tqdm import tqdm
uiq,suq = [],[]
uiq_tok, suq_tok = [],[]
for i in tqdm(range(10000)):
    user_id = train_dataset.usermap[i]
    data = train_dataset.data
    meta_name_dict = train_dataset.meta_name_dict
    tokenizer = train_dataset.tokenizer
    similar_users = train_dataset.similar_users

    user_item_query = train_dataset._format_item_list_query(user_id, data, meta_name_dict)
    similar_users_query = train_dataset._format_similar_users_query(user_id, data, meta_name_dict, similar_users)
    ui_tok = tokenizer(user_item_query, return_tensors='np')
    su_tok = tokenizer(similar_users_query, return_tensors='np')
    uiq.append(user_item_query)
    suq.append(similar_users_query)
    uiq_tok.append(ui_tok['input_ids'].shape[1])
    suq_tok.append(su_tok['input_ids'].shape[1])


100%|██████████| 10000/10000 [00:15<00:00, 657.07it/s]


In [38]:
import numpy as np

np.percentile(uiq_tok, q=[0,25,50,75,99,100]), np.percentile(suq_tok, q=[0,25,50,75,99,100])

(array([105., 150., 224., 407., 699., 898.]),
 array([106.  , 113.  , 122.  , 159.  , 265.01, 584.  ]))